# CS229, Fall 2017
## Problem Set 4: EM, DL & RL
### 1. Neural Networks: MNIST image classification

#### (a)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gc
import sys

In [2]:
def nn_test(data, labels, params):
    h, output, cost = forward_prop(data, labels, params)
    accuracy = compute_accuracy(output, labels)
    return accuracy


def compute_accuracy(output, labels):
    accuracy = (np.argmax(output, axis=1) == np.argmax(
        labels, axis=1)).sum() * 1. / labels.shape[0]
    return accuracy


def one_hot_labels(labels):
    one_hot_labels = np.zeros((labels.size, 10))
    one_hot_labels[np.arange(labels.size), labels.astype(int)] = 1
    return one_hot_labels

In [3]:
def readData(images_file, labels_file):
    x = np.loadtxt(images_file, delimiter=',')
    y = np.loadtxt(labels_file, delimiter=',')
    return x, y


def softmax(x):
    """
    Compute softmax function for input. 
    Use tricks from previous assignment to avoid overflow
    """
    # YOUR CODE HERE
    s = x / np.sum(np.exp(x))
    # END YOUR CODE
    return s


def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    """
    # YOUR CODE HERE
    s = 1 / (1 + np.exp(-x))
    # END YOUR CODE
    return s


def forward_prop(data, labels, params):
    """
    return hidder layer, output(softmax) layer and loss
    """
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']

    # YOUR CODE HERE
    '''
    data: a n by 1 vector, where n = 784
    W1: a k by n matrix, where k = 300 is the number of hidden units
    W2: a 10 by k matrix
    '''
    # 1. Input layer to the hidden layer
    h = sigmoid(W1 @ data + b1)

    # 2. Hidden layer to output layer
    y = softmax(W2 @ h + b2)

    # 3. Compute the cost
    cost = -np.sum(labels * np.log(y))

    # END YOUR CODE
    return h, y, cost

##### Backpropagation

For a neural network in a more general form, we define the following notations:  
* $X^{L}$: the input of layer L, a n by 1 matrix (vector)  
* $W^{L}$: the weight between layer $L$ and layer $L+1$, a m by n matrix (where m is the number of the neurons in the $L+1 $ layer and n is the number of neurons in the $L$ layer)
* $Z^{L}=W^{L}X^L$: the output of the $L$ layer, a m by 1 matrix
* $A^{L}=f(Z^{L})$: the activation output of the $L$ layer, a m by 1 matrix

Backpropagation involves 3 steps:
1. Calculate the error term of the output layer directly
2. Calculate the error term of all the other layers by using the error term of its later layer (back propagate the error)
3. Calculate the derivates using the error term

**Note: the multivariate chain rule**  
For function $z=f(x,y)$, where $x,y$ is functions of $t$, therefore we have:
$$
\frac{\partial f}{\partial t} = \frac{\partial f}{\partial x}\frac{\partial x}{\partial t} + \frac{\partial f}{\partial y}\frac{\partial y}{\partial t}
$$

###### Step 1

We define the error term as:
$$
\delta^{L} = \frac{\partial \ell}{\partial Z^{L}}
$$
where $\ell$ is the loss function. By the definition of gradient, we know that $\delta^L$ is a K by 1 matrix.
For the error term of the output layer, we begin by examing one element in that error term:
\begin{align*}
\delta_i^{L} &= \frac{\partial \ell}{\partial Z_i^{L}}\\
&= \sum_{k}\frac{\partial \ell}{\partial A_k^{L}} \frac{\partial A_k^{L}}{Z_i^{L}}\\
&= \frac{\partial \ell}{\partial A_i^L}\frac{\partial A_i^L}{Z_i^L}
\end{align*}
We take this question as an example, in this problem, the loss function $\ell$ is the cross-entropy, which has the following form:
$$
\ell = -\sum_{k=1}^Ky_k\log \hat{y}_k
$$
and also $\hat{y}_k=A_k^L=f(Z_1^L,\dots,Z_k^L,\dots, Z_K^L)$ where $f$ is the softmax function. Therefore, the above equation is:
\begin{align*}
\delta_i^{L} &= \frac{\partial \ell}{\partial A_i^L}\frac{\partial A_i^L}{Z_i^L}\\
&= -\sum_{k}^K\frac{y_k}{\hat{y}_k}\frac{\partial \hat{y}_k}{\partial Z_i^L}
\end{align*}

For the softmax function, we have:
\begin{align*}
\frac{\partial \hat{y}_k}{\partial Z_i^L}=\left\{
    \begin{matrix}
    \frac{e^{z_j}\sum_{k=1}^Ke^{z_k}-e^{2z_j}}{(\sum_{k=1}^Ke^{z_k})^2}=\frac{e^{z_j}}{\sum_{k=1}^Ke^{z_k}}-\frac{e^{z_j}}{\sum_{k=1}^Ke^{z_k}}\frac{e^{z_j}}{\sum_{k=1}^Ke^{z_k}}={\hat{y}_k(1-\hat{y}_k)}&i==k\\
    -\frac{e^{z_j+z_k}}{(\sum_{k=1}^Ke^{z_k})^2}=-\hat{y}_k\hat{y}_i&i\ne k
    \end{matrix}
\right.
\end{align*}

The complete matrix form is:
\begin{align*}
\delta^L&=\frac{\partial \ell}{\partial Z^L}\\
&= J_zA\nabla_A\ell
\end{align*}
where $J_zA$ is a K by K Jacobian matrix ($J_zA_{ik}=\frac{\partial \hat{y}_k}{\partial Z_i^L}$) and $\nabla_A\ell$ is a K by 1 vector.

###### Step 2

For all the other layers except the output layer, we have the corresponding error term as:
\begin{align*}
\delta_i^{L-1} &= \frac{\partial\ell}{\partial Z_i^{L-1}}\\
&= \sum_{k=1}^K\frac{\partial \ell}{\partial Z_k^{L}}\frac{\partial Z_k^{L}}{\partial Z_i^{L-1}}\\
&= \sum_{k=1}^K\frac{\partial \ell}{\partial Z_k^{L}}\frac{\partial Z_k^{L}}{\partial A_i^{L-1}}\frac{\partial A_i^{L-1}}{\partial Z_i^{L-1}}\\
&= \sum_{k=1}^K\delta_k^{L}W_{ki}^{L-1}\sigma(Z_i^{L-1})(1-\sigma(Z_i^{L-1}))
\end{align*}
where $\sigma(x)$ is the sigmoid function and we use the fact that $\sigma'=\sigma(1-\sigma)$

The complete matrix form is:
\begin{align*}
\delta^{L-1}={\delta^L}^TW^{L-1}\odot\bigg(\sigma(Z^{L-1})(1-\sigma(Z^{L-1}))\bigg)
\end{align*}

###### Step 3

The derivate can be calculated by:
\begin{align*}
\frac{\partial \ell}{\partial W_{ij}^{L-1}}&=\sum_{k=1}^K\frac{\partial \ell}{\partial Z_k^L}\frac{\partial Z_k^L}{\partial W_{ij}^{L-1}}\\
&=\frac{\partial \ell}{\partial Z_i^L}\frac{\partial Z_i^L}{\partial W_{ij}^{L-1}}\\
&=\delta_i^LA_j^{L-1}
\end{align*}

The complete matrix form is:
\begin{align*}
H_{W^{L-1}}\ell=\delta^L{A^{L-1}}^T
\end{align*}

In [17]:
def backward_prop(data, labels, params):
    """
    return gradient of parameters
    """
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']

    # YOUR CODE HERE
    # First, we do a forward propagation
    h, y, cost = forward_prop(data, labels, params)

    # Second, we calculate all error
    nabla_loss = labels / y
    nabla_softmax = -y @ y.T
    nabla_softmax -= np.diag(np.diag(nabla_softmax))
    nabla_softmax += np.diag((y * (1 - y)).reshape(-1))
    delta_2 = nabla_softmax @ nabla_loss

    # Propagate backward
    delta_1 = (delta_2.T @ W2).reshape([-1, 1]) * (h * (1 - h))

    # Get the gradient
    gradW1 = delta_1.reshape([-1, 1]) @ data.reshape([1, -1])
    gradb1 = delta_1
    gradW2 = delta_2.reshape([-1, 1]) @ h.reshape([1, -1])
    gradb2 = delta_2
    # END YOUR CODE

    grad = {}
    grad['W1'] = gradW1
    grad['W2'] = gradW2
    grad['b1'] = gradb1
    grad['b2'] = gradb2

    return grad


def total_error(output, label):
    # output is a m by 10 matrix, where m is the number of training examples
    return np.sum(label * np.log(output)) / output.shape[0]

In [18]:
def nn_train(trainData, trainLabels, devData, devLabels):
    print('Training begin')
    (m, n) = trainData.shape
    num_hidden = 300
    learning_rate = 5
    params = {}

    # YOUR CODE HERE
    # Initialize the parameter
    params['b1'] = np.zeros([num_hidden,1])
    params['b2'] = np.zeros([10,1])
    params['W1'] = np.random.randn(num_hidden, n)
    params['W2'] = np.random.randn(10, num_hidden)
    
    B = 1000 # Mini batch size
    iteration_per_epoch = m / 1000
    epoch_round = 30
    error_history = [0] * epoch_round
    for i in range(epoch_round):
        print('Epoch %d begin' % (i + 1))
        for j in range(int(iteration_per_epoch)):
            # One batch
            batch_data = np.array(trainData[j * B:(j + 1) * B])
            batch_label = np.array(trainLabels[j * B:(j + 1) * B])
            grad_total = {k:np.zeros(params[k].shape) for k in params}
            for data, label in zip(batch_data, batch_label):
                grad = backward_prop(data.reshape(-1,1), label.reshape(-1,1), params)
                for k in grad:
                    grad_total[k] += grad[k]
            # Update parameter
            for k in params:
                params[k] -= (learning_rate / B) * grad_total[k]
        # Calculate error averaged over the entire data set
        for data, label in zip(trainData, trainLabels):
            _, _, loss = forward_prop(data, label, params)
            error_history[i] += loss
        print('Epoch %d end, error %f' % (i + 1, error_history[i]))
    # END YOUR CODE

    return params

In [6]:
np.random.seed(100)
print('Begin loading data')
trainData, trainLabels = readData(
    './data/images_train.csv', './data/labels_train.csv')
trainLabels = one_hot_labels(trainLabels)
p = np.random.permutation(60000)
trainData = trainData[p, :]
trainLabels = trainLabels[p, :]

devData = trainData[0:10000, :]
devLabels = trainLabels[0:10000, :]
trainData = trainData[10000:, :]
trainLabels = trainLabels[10000:, :]

mean = np.mean(trainData)
std = np.std(trainData)
trainData = (trainData - mean) / std
devData = (devData - mean) / std

testData, testLabels = readData(
    './data/images_test.csv', './data/labels_test.csv')
testLabels = one_hot_labels(testLabels)
testData = (testData - mean) / std
print('Loading end')

Begin loading data
Loading end


In [19]:
print('Begin training')
params = nn_train(trainData, trainLabels, devData, devLabels)

readyForTesting = False
if readyForTesting:
        accuracy = nn_test(testData, testLabels, params)
        print('Test accuracy: %f' % accuracy)

Begin training
Training begin
Epoch 1 begin


/home/cloud/.local/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in log
/home/cloud/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in exp
/home/cloud/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]
/home/cloud/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in subtract
/home/cloud/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in matmul


KeyboardInterrupt: 

In [24]:
dir()

['In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'autopep8',
 'backward_prop',
 'compute_accuracy',
 'devData',
 'devLabels',
 'exit',
 'forward_prop',
 'gc',
 'get_ipython',
 'json',
 'mean',
 'nn_test',
 'nn_train',
 'np',
 'one_hot_labels',
 'p',
 'plt',
 'quit',
 'readData',
 'sigmoid',
 'softmax',
 'std',
 'testData',
 'testLabels',
 'total_error',
 'trainData',
 'trainLabels']

In [28]:
del total_error
gc.collect()

21241